## This script plots altair plots by custom features

In [2]:
import numpy as np
import csv
# from matplotlib.colors import LinearSegmentedColormap, to_hex, rgb2hex
import pandas as pd
import altair as alt
import pickle
# import argparse
# import scipy.stats

import sys
sys.path.append("/Users/victoriagao/Documents/MSc/rotation/GW_lab/altair-themes")
if True:  # In order to bypass isort when saving
    import altairThemes

### Read in spot region info

In [17]:
# Read in filtered connected component set
# with open('/Users/victoriagao/local_docs/NEST/stored_variables/PDAC_connected_subtypes/PDAC_64630_filtered_connected_subtype_components.pkl', 'rb') as file:
#     filtered_connected_components = pickle.load(file)

with open('/Users/victoriagao/local_docs/NEST/stored_variables/filtered_connected_components.pkl', 'rb') as file:
    filtered_connected_components = pickle.load(file)

# Read in combined_barcode_list info required for plotting
# with open('/Users/victoriagao/local_docs/NEST/stored_variables/PDAC_connected_subtypes/PDAC_64630_combined_barcode_list.pkl', 'rb') as file:
#     combined_barcode_list = pickle.load(file)

with open('/Users/victoriagao/local_docs/NEST/stored_variables/cell_barcode.pkl', 'rb') as file:
    cell_barcode = pickle.load(file)


# Load coordinates
coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/coordinates.npy")

# Load pathologist's label
data_name = 'PDAC_64630'
if data_name == 'PDAC_64630':
    pathologist_label_file='/Users/victoriagao/local_docs/NEST/input/pathologist_annot/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
    pathologist_label=[]
    with open(pathologist_label_file) as file:
        csv_file = csv.reader(file, delimiter=",")
        for line in csv_file:
            pathologist_label.append(line)	
    	
    barcode_type=dict() # record the type (annotation) of each spot (barcode)
    for i in range (1, len(pathologist_label)):
        barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]

In [19]:
# Make barcode_info if DOES NOT already have combined_barcode_list
i=0
barcode_serial = dict()
for cell_code in cell_barcode:
    barcode_serial[cell_code] = i
    i = i + 1

i=0
barcode_info=[]
barcode_info.append(["cell_code", "x-coordinate", "y-coordinate", "component", "color", "barcode_number"]) # add title
for cell_code in cell_barcode:
    barcode_info.append([cell_code, coordinates[i,0],coordinates[i,1], 0, 0, 0, 0]) # last entries will hold the component number later
    i=i+1


barcode_coordinate_dict = {item[0]: [item[1], item[2]] for item in barcode_info}

combined_barcode = {}
for key in barcode_type:
    if key in barcode_coordinate_dict:
        combined_barcode[key] = [barcode_type[key]] + barcode_coordinate_dict[key]


combined_barcode_list = []

for key, values in combined_barcode.items():
    temp_dict = {'barcode': key, 'type': values[0], 'x': values[1], 'y': values[2]}
    combined_barcode_list.append(temp_dict)

In [20]:
barcode_to_set = {}
for idx, s in enumerate(filtered_connected_components):
    for barcode in s:
        barcode_to_set[barcode] = idx

In [21]:
barcode_to_set

{'CCATGGCCCTTGTACC-1': 0,
 'TACTGGACAGCTCGGC-1': 0,
 'TATGGCCCGGCCTCGC-1': 0,
 'AAGTAGTGACGCGAGG-1': 0,
 'GTTACAATTGGTGACG-1': 0,
 'CATACCCGTACCCAGT-1': 0,
 'GGTTAGTTACGGCGCC-1': 0,
 'CCGCGTAGGTAAGGGC-1': 0,
 'GAAACTCGTGCGATGC-1': 0,
 'GTTTCATATCGTCGCT-1': 0,
 'TTGGACATGTGGCTTA-1': 0,
 'ATTCCCACATAAACAA-1': 0,
 'CCGGGTTCGAGGTTAC-1': 0,
 'CTAACCGCGCGCCCGT-1': 0,
 'GGTGCTGATCACAAAG-1': 0,
 'GAAATATCACCATCAG-1': 0,
 'GCAGGACTATAGAATA-1': 0,
 'AACAATTACTCTACGC-1': 0,
 'GGGAACGGGAGGTTAG-1': 0,
 'CGTGCATTGTCGACGC-1': 0,
 'GCGTCTCTGCATTGGG-1': 0,
 'CAGGCCGTTTGGGTGT-1': 0,
 'TCCCTGGCTCGCTGGA-1': 0,
 'AGATACCGGTGTTCAC-1': 0,
 'ACAAGCAGTGCCTAGC-1': 0,
 'GGCATACAGGTAGCGG-1': 0,
 'CATGCCAACTCGCAAA-1': 0,
 'GAGGCTATCAAAGTCG-1': 0,
 'TCCTCTCCAGTTGTCC-1': 0,
 'CATAGCCGCCCGGGAT-1': 0,
 'TGTAGTGATCTATAAT-1': 0,
 'GACGCTTGCTTCTAAA-1': 0,
 'ACGCGCTACACAGGGT-1': 0,
 'TCACATCTTATCTGAT-1': 0,
 'GGCGCAGGACATCTTC-1': 0,
 'GTACTCCCTTATCGCT-1': 0,
 'TCGGAATGCGCTCTGA-1': 0,
 'TATCTACAGAGGTAAT-1': 0,
 'CGACAGTTCG

In [22]:
set1 = altairThemes.get_colour_scheme("Set1", 53)
data = []

for idx, barcode_set in enumerate(filtered_connected_components):
    for barcode in barcode_set:
        for entry in combined_barcode_list:
            if entry['barcode'] == barcode:
                data.append({
                    'Barcode': barcode,
                    'Type': entry['type'],
                    'X': entry['x'],
                    'Y': -entry['y'],
                    'Set': f'{idx + 1}',
                    # 'Color': set1[idx % len(set1)]
                })

df = pd.DataFrame(data)

In [23]:
df

,Barcode,Type,X,Y,Set
0,CCATGGCCCTTGTACC-1,tumor,7559,-12621,1
1,TACTGGACAGCTCGGC-1,tumor,8076,-12321,1
2,TATGGCCCGGCCTCGC-1,tumor,7210,-11829,1
3,AAGTAGTGACGCGAGG-1,tumor,7728,-11727,1
4,GTTACAATTGGTGACG-1,tumor,8418,-11526,1
...,...,...,...,...,...
1393,GTCACTCTCCAAATCT-1,tumor,6001,-11734,20
1394,CGCAATTACTTTCGGT-1,tumor,6000,-11536,20
1395,TAAATGAATCCGTTTC-1,tumor,6002,-11933,20
1396,TCCTTTAAATCCGCTT-1,Artifact,11673,-5657,21


### Plot by subtype regions (colors are diff regions)

In [22]:
# base = alt.Chart(df)

# # Create scatter plot for tumor
# tumor_plot = base.mark_square(filled=True, opacity = 1).encode(
#     # x='X:Q',
#     # y='Y:Q',
#     alt.X('X', scale=alt.Scale(zero=False)),
#     alt.Y('Y', scale=alt.Scale(zero=False)),
#     # color=alt.value('black'),
#     #color=alt.Color('Set:N', scale=alt.Scale(range=set1)),
#     color=alt.Color('Set:N', scale=alt.Scale(scheme='category20')),
#     # color=alt.Color('Hill_num:Q', scale=alt.Scale(range=color_scale_range), title='Hill Number'),
#     tooltip=['Set:N', 'Type:N','Hill_num:Q']
# ).transform_filter(
#     alt.datum.Type == 'ClassicB'
# )

# # Create scatter plot for non-tumor spots
# stroma_plot = base.mark_circle(filled=False, opacity = 1, strokeWidth=0.5).encode(
#     # x='X:Q',
#     # y='Y:Q',
#     alt.X('X', scale=alt.Scale(zero=False)),
#     alt.Y('Y', scale=alt.Scale(zero=False)),
#     color=alt.value('black'),
#     # color=alt.Color('Set:N', scale=alt.Scale(range=set1)),
#     tooltip=['Set:N', 'Type:N','X:Q', 'Y:Q']
# ).transform_filter(
#     alt.datum.Type != 'BasalB'
# )

# final_plot = tumor_plot + stroma_plot

In [29]:
# Create the Altair plot
final_plot = alt.Chart(df).mark_point(filled=True).encode(
    x='X',
    y='Y',
    color='Set:N',
    shape='Type:N',
    tooltip=['Type:N','Set:N']
)

### Plot by Freq

In [37]:
###### Make plotting df

# Load stored hill number dictionary
with open('/Users/victoriagao/local_docs/NEST/stored_variables/64630_1.24_results/64630_LR_freq_TS_interact.pickle', 'rb') as file:
    LR_freq_dict = pickle.load(file)

# Add a new column for freq
df['Set'] = df['Set'].astype(int)
df['LR_freq'] = df['Set'].map(LR_freq_dict)

In [38]:
df

,Barcode,Type,X,Y,Set,Hill_num,LR_freq
0,CCATGGCCCTTGTACC-1,tumor,7559,-12621,1,0.684562,20
1,TACTGGACAGCTCGGC-1,tumor,8076,-12321,1,0.684562,20
2,TATGGCCCGGCCTCGC-1,tumor,7210,-11829,1,0.684562,20
3,AAGTAGTGACGCGAGG-1,tumor,7728,-11727,1,0.684562,20
4,GTTACAATTGGTGACG-1,tumor,8418,-11526,1,0.684562,20
...,...,...,...,...,...,...,...
1393,GTCACTCTCCAAATCT-1,tumor,6001,-11734,20,0.000000,4
1394,CGCAATTACTTTCGGT-1,tumor,6000,-11536,20,0.000000,4
1395,TAAATGAATCCGTTTC-1,tumor,6002,-11933,20,0.000000,4
1396,TCCTTTAAATCCGCTT-1,Artifact,11673,-5657,21,0.000000,0


In [ ]:
###### This chunk plots regions based on Hill number

# color_scale_range = ['rgb(255, 0, 0)', 'rgb(128, 0, 0)']  # Red to dark red gradient

base = alt.Chart(df)


# Create scatter plot for tumor
tumor_plot = base.mark_square(filled=True, opacity = 1).encode(
    # x='X:Q',
    # y='Y:Q',
    alt.X('X', scale=alt.Scale(zero=False)),
    alt.Y('Y', scale=alt.Scale(zero=False)),
    # color=alt.value('black'),
    # color=alt.Color('Set:N', scale=alt.Scale(range=set1)),
    color=alt.Color('LR_freq:Q', scale=alt.Scale(scheme="magma"), title='CCC Freq'),
    tooltip=['Set:N', 'Type:N','LR_freq:Q']
).transform_filter(
    alt.datum.Type == 'tumor'
)

# Create scatter plot for non-tumor spots
other_plot = base.mark_circle(filled=False, opacity = 1,strokeWidth=0.3).encode(
    # x='X:Q',
    # y='Y:Q',
    alt.X('X', scale=alt.Scale(zero=False)),
    alt.Y('Y', scale=alt.Scale(zero=False)),
    color=alt.value('black'),
    # color=alt.Color('Set:N', scale=alt.Scale(range=set1)),
    tooltip=['Set:N', 'Type:N','X:Q', 'Y:Q']
).transform_filter(
    alt.datum.Type != 'tumor'
)

In [ ]:
final_plot = tumor_plot + other_plot

### Plot by Hill number

In [32]:
###### Make plotting df

# Load stored hill number dictionary
with open('/Users/victoriagao/local_docs/NEST/stored_variables/64630_hill_withinTumor.pickle', 'rb') as file:
    hill_number_dict = pickle.load(file)

# Add a new column for Hill number(diversity index)
df['Set'] = df['Set'].astype(int)
df['Hill_num'] = df['Set'].map(hill_number_dict)

In [33]:
df

,Barcode,Type,X,Y,Set,Hill_num
0,CCATGGCCCTTGTACC-1,tumor,7559,-12621,1,0.684562
1,TACTGGACAGCTCGGC-1,tumor,8076,-12321,1,0.684562
2,TATGGCCCGGCCTCGC-1,tumor,7210,-11829,1,0.684562
3,AAGTAGTGACGCGAGG-1,tumor,7728,-11727,1,0.684562
4,GTTACAATTGGTGACG-1,tumor,8418,-11526,1,0.684562
...,...,...,...,...,...,...
1393,GTCACTCTCCAAATCT-1,tumor,6001,-11734,20,0.000000
1394,CGCAATTACTTTCGGT-1,tumor,6000,-11536,20,0.000000
1395,TAAATGAATCCGTTTC-1,tumor,6002,-11933,20,0.000000
1396,TCCTTTAAATCCGCTT-1,Artifact,11673,-5657,21,0.000000


In [34]:
###### This chunk plots regions based on Hill number

# color_scale_range = ['rgb(255, 0, 0)', 'rgb(128, 0, 0)']  # Red to dark red gradient

base = alt.Chart(df)


# Create scatter plot for tumor
tumor_plot = base.mark_square(filled=True, opacity = 1).encode(
    # x='X:Q',
    # y='Y:Q',
    alt.X('X', scale=alt.Scale(zero=False)),
    alt.Y('Y', scale=alt.Scale(zero=False)),
    # color=alt.value('black'),
    # color=alt.Color('Set:N', scale=alt.Scale(range=set1)),
    color=alt.Color('Hill_num:Q', scale=alt.Scale(scheme="magma"), title='Hill Number'),
    tooltip=['Set:N', 'Type:N','Hill_num:Q']
).transform_filter(
    alt.datum.Type == 'tumor'
)

# Create scatter plot for non-tumor spots
other_plot = base.mark_circle(filled=False, opacity = 1,strokeWidth=0.3).encode(
    # x='X:Q',
    # y='Y:Q',
    alt.X('X', scale=alt.Scale(zero=False)),
    alt.Y('Y', scale=alt.Scale(zero=False)),
    color=alt.value('black'),
    # color=alt.Color('Set:N', scale=alt.Scale(range=set1)),
    tooltip=['Set:N', 'Type:N','X:Q', 'Y:Q']
).transform_filter(
    alt.datum.Type != 'tumor'
)

In [35]:
final_plot = tumor_plot + other_plot


### Save

In [36]:
save_path = '/Users/victoriagao/local_docs/NEST/output/visualization_html/'
final_plot.save(save_path+'NEW_altair_plot.html')

/Users/victoriagao/anaconda3/envs/altair/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
